# Cis-eQTL Peak Overlap

**Created**: 14 December 2021

## Environment

In [1]:
library(tidyverse)
library(data.table)
library(GenomicRanges)

setwd("~/eQTL_pQTL_Characterization")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: stats4

Loading required package

## Load Data

In [2]:
cis.eqtl <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/cisqtl_all_significant.rds")

In [3]:
head(cis.eqtl)

snps,gene,statistic,pvalue,beta,se,chr,SNPpos,TSS,threshold
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>
rs4987343,ENSG00000000457,-4.152142,3.720893e-05,-0.04958163,0.01194122,1,169699715,169894267,3.873864e-05
rs4987340,ENSG00000000457,-4.240369,2.546937e-05,-0.05063396,0.01194093,1,169700002,169894267,3.873864e-05
rs2298899,ENSG00000000457,4.176920,3.365090e-05,0.05505924,0.01318178,1,169700591,169894267,3.873864e-05
rs4987328,ENSG00000000457,4.252010,2.432691e-05,0.05597519,0.01316441,1,169701315,169894267,3.873864e-05
rs4987327,ENSG00000000457,-4.218194,2.799604e-05,-0.05023696,0.01190959,1,169701799,169894267,3.873864e-05
rs4987326,ENSG00000000457,-4.218194,2.799604e-05,-0.05023696,0.01190959,1,169701800,169894267,3.873864e-05


In [4]:
geno <- fread("~/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.bim")
colnames(geno) <- c("chr", "snp", "cM", "pos", "minor", "major")

In [5]:
head(geno)

chr,snp,cM,pos,minor,major
<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,rs3131972,0,817341,A,G
1,rs546843995,0,818053,0,G
1,rs553916047,0,818359,0,A
1,1:818740_T_C,0,818740,T,C
1,rs145604921,0,819378,0,C
1,rs535256652,0,821053,0,T


In [9]:
cis.eqtl.conditional <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/conditionalanalysis/conditional_eQTL_results_final.rds")
cis.eqtl.conditional <- merge(cis.eqtl.conditional, geno, by.x="SNP", by.y="snp") %>%
    dplyr::select(snps=SNP, gene=Gene, pvalue, beta=eQTL_beta, se=eQTL_SE, chr, SNPpos=pos) %>%
    dplyr::mutate(pvalue=as.numeric(pvalue)) %>%
    dplyr::mutate(beta=as.numeric(beta)) %>%
    dplyr::mutate(se=as.numeric(se)) %>%
    dplyr::mutate(chr=as.numeric(chr))

In [10]:
head(cis.eqtl.conditional)

,snps,gene,pvalue,beta,se,chr,SNPpos
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,rs10000507,ENSG00000164305,1.194703e-05,-0.05880549,0.013312181,4,184698127
2,rs10000850,ENSG00000128039,7.576675e-43,-0.64265343,0.043284640,4,55634211
3,rs1000137,ENSG00000272839,1.261994e-13,-0.17162745,0.022657505,7,157851460
4,rs10001795,ENSG00000121892,2.088410e-05,-0.03630321,0.008460434,4,40366982
5,rs10004630,ENSG00000170390,1.804495e-17,0.12054523,0.013573487,4,150065588
6,rs10007349,ENSG00000038210,1.368337e-07,0.11265677,0.021188828,4,25224916


In [11]:
da.peaks <- read.csv("~/gains_team282/epigenetics/calderon_et_al/analysis/atac_seq/da_peak_set.csv", row.names=1)

In [12]:
head(da.peaks)

,Contrast,Cell.Type,Peak.ID,Peak.Chr,Peak.Start,Peak.End,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,13:50244295-50246227,13,50244295,50246227,4.068530,1.9922172,9.547057,7.190173e-17,4.418153e-11,25.19917
2,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,13:79450967-79452794,13,79450967,79452794,3.720693,2.0688902,9.146720,7.240018e-16,2.224391e-10,23.23470
3,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,9:99470831-99473127,9,99470831,99473127,3.924045,1.2458540,9.056953,1.211705e-15,2.481858e-10,22.79560
4,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,16:19293486-19294933,16,19293486,19294933,5.314531,0.3815736,8.939779,2.369063e-15,3.639301e-10,22.22338
5,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,10:90977043-90978869,10,90977043,90978869,3.431728,1.0461273,8.440246,4.027307e-14,4.949327e-09,19.79878
6,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,7:140375586-140377343,7,140375586,140377343,3.786718,2.0515648,8.237938,1.252067e-13,1.282265e-08,18.82526


In [13]:
de.genes <- read.csv("~/gains_team282/epigenetics/calderon_et_al/analysis/rna_seq/de_genes_set.csv", row.names=1)

In [14]:
head(de.genes)

,Contrast,Cell.Type,Gene.ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000120129,-9.903447,5.807615,-8.916094,4.638292e-15,1.607967e-10,22.31871
2,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000067082,-4.802029,9.347232,-8.839756,7.074361e-15,1.607967e-10,21.94573
3,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000157514,-9.019418,5.049123,-8.433398,6.600765e-14,1.000214e-09,19.96921
4,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000170345,-10.608820,4.083466,-8.368503,9.407246e-14,1.069110e-09,19.65517
5,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000234572,-7.755181,-4.424018,-8.084770,4.389632e-13,3.990966e-09,18.28843
6,Bulk_B_Stimulated-Bulk_B_Baseline,Bulk_B,ENSG00000251309,-7.769301,-4.515273,-7.801246,2.014435e-12,1.526237e-08,16.93438


## Overlap of Sentinel cis-eQTL

First, I'll start by testing the lead SNP for each gene for overlap with DA peaks. These peaks are derived from the initial pass analysis of cis-eQTL (not the conditional analysis). A sentinel SNP is the SNP with the highest p-value for a given location, representing the SNP with the highest evidence for association with the expression of the cognate gene.

In [15]:
da.peak.ranges = GRanges(
    seqnames=da.peaks$Peak.Chr,
    ranges=IRanges(
        start=da.peaks$Peak.Start,
        end=da.peaks$Peak.End
    ),
    strand=rep("*", nrow(da.peaks))
)
mcols(da.peak.ranges) <- da.peaks[,c("Peak.ID", "Cell.Type", "logFC", "adj.P.Val")]

In [16]:
da.peak.ranges

GRanges object with 171830 ranges and 4 metadata columns:
           seqnames              ranges strand |                Peak.ID
              <Rle>           <IRanges>  <Rle> |            <character>
       [1]       13   50244295-50246227      * |   13:50244295-50246227
       [2]       13   79450967-79452794      * |   13:79450967-79452794
       [3]        9   99470831-99473127      * |    9:99470831-99473127
       [4]       16   19293486-19294933      * |   16:19293486-19294933
       [5]       10   90977043-90978869      * |   10:90977043-90978869
       ...      ...                 ...    ... .                    ...
  [171826]       13 112116343-112116882      * | 13:112116343-112116882
  [171827]       16   67758899-67760054      * |   16:67758899-67760054
  [171828]       16   85514268-85516322      * |   16:85514268-85516322
  [171829]        5   94983466-94984498      * |    5:94983466-94984498
  [171830]       22   26614942-26615897      * |   22:26614942-26615897
      

In [17]:
cis.eqtl.sentinel <- cis.eqtl %>%
    dplyr::group_by(gene) %>%
    dplyr::slice_min(order_by=pvalue, n=1)

cis.eqtl.sentinel.ranges = GRanges(
    seqnames=cis.eqtl.sentinel$chr,
    ranges=IRanges(
        start=cis.eqtl.sentinel$SNPpos,
        end=cis.eqtl.sentinel$SNPpos
    ),
    strand=rep("*", nrow(cis.eqtl.sentinel))
)
mcols(cis.eqtl.sentinel.ranges) <- cis.eqtl.sentinel[,c("gene", "TSS", "beta", "pvalue")]

In [18]:
cis.eqtl.sentinel.ranges

GRanges object with 17286 ranges and 4 metadata columns:
          seqnames    ranges strand |            gene       TSS       beta
             <Rle> <IRanges>  <Rle> |     <character> <integer>  <numeric>
      [1]        1 169776143      * | ENSG00000000457 169894267  0.0498587
      [2]        1 169776312      * | ENSG00000000457 169894267  0.0498587
      [3]        1 169749095      * | ENSG00000000460 169662007 -0.0721270
      [4]        1 196708768      * | ENSG00000000971 196652043  0.1713537
      [5]        1 196726612      * | ENSG00000000971 196652043  0.1713537
      ...      ...       ...    ... .             ...       ...        ...
  [17282]       11  78283443      * | ENSG00000288538  78223815 -0.2467764
  [17283]       11  78284097      * | ENSG00000288538  78223815 -0.2467764
  [17284]        1 186609504      * | ENSG00000288562 186624700  0.1080522
  [17285]        1 186617709      * | ENSG00000288562 186624700  0.1080522
  [17286]        1  43916852      * | ENSG0

In [19]:
overlaps <- findOverlaps(cis.eqtl.sentinel.ranges, da.peak.ranges)

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 23
  - in 'y': X, KI270726.1, KI270742.1, KI270717.1, GL000008.2, KI270734.1, KI270706.1, GL000221.1, KI270731.1, KI270749.1, KI270733.1, GL000220.1, GL000009.2, GL000194.1, KI270709.1, KI270750.1, GL000195.1, KI270727.1, KI270748.1, Y, GL000224.1, KI270744.1, GL000216.2, GL000219.1, KI270721.1, KI270719.1, KI270712.1, KI270725.1, KI270722.1, KI270707.1, KI270538.1, KI270442.1, KI270708.1, KI270732.1, GL000205.2, KI270737.1, KI270735.1, GL000214.1, KI270438.1, GL000218.1, KI270746.1, KI270736.1
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”


In [20]:
overlap.set <- dplyr::bind_cols(
    as.data.frame(cis.eqtl.sentinel.ranges[overlaps@from, ]) %>%
        dplyr::select(SNP.Chr=seqnames, SNP.Pos=start, eQTL.Gene=gene, eQTL.TSS=TSS, eQTL.Beta=beta, eQTL.P=pvalue),
    as.data.frame(da.peak.ranges[overlaps@to, ]) %>%
        dplyr::select(Peak.ID=Peak.ID, Peak.Chr=seqnames, Peak.Start=start, Peak.End=end, Peak.Width=width, Peak.Cell.Type=Cell.Type, Peak.Log.FC=logFC, Peak.P=adj.P.Val)
)

In [21]:
length(unique(overlap.set$Peak.ID))

[1] 485

In [22]:
length(unique(paste0(overlap.set$SNP.Chr, ":", overlap.set$SNP.Pos)))

[1] 582

In [23]:
length(unique(overlap.set$eQTL.Gene))

[1] 499

## Overlap of All Significant cis-eQTL

I'll test all significant SNPs for each gene for overlap with DA peaks. These peaks are derived from the initial pass analysis of cis-eQTL (not the conditional analysis).

In [24]:
da.peak.ranges = GRanges(
    seqnames=da.peaks$Peak.Chr,
    ranges=IRanges(
        start=da.peaks$Peak.Start,
        end=da.peaks$Peak.End
    ),
    strand=rep("*", nrow(da.peaks))
)
mcols(da.peak.ranges) <- da.peaks[,c("Peak.ID", "Cell.Type", "logFC", "adj.P.Val")]

In [25]:
da.peak.ranges

GRanges object with 171830 ranges and 4 metadata columns:
           seqnames              ranges strand |                Peak.ID
              <Rle>           <IRanges>  <Rle> |            <character>
       [1]       13   50244295-50246227      * |   13:50244295-50246227
       [2]       13   79450967-79452794      * |   13:79450967-79452794
       [3]        9   99470831-99473127      * |    9:99470831-99473127
       [4]       16   19293486-19294933      * |   16:19293486-19294933
       [5]       10   90977043-90978869      * |   10:90977043-90978869
       ...      ...                 ...    ... .                    ...
  [171826]       13 112116343-112116882      * | 13:112116343-112116882
  [171827]       16   67758899-67760054      * |   16:67758899-67760054
  [171828]       16   85514268-85516322      * |   16:85514268-85516322
  [171829]        5   94983466-94984498      * |    5:94983466-94984498
  [171830]       22   26614942-26615897      * |   22:26614942-26615897
      

In [26]:
cis.eqtl.ranges = GRanges(
    seqnames=cis.eqtl$chr,
    ranges=IRanges(
        start=cis.eqtl$SNPpos,
        end=cis.eqtl$SNPpos
    ),
    strand=rep("*", nrow(cis.eqtl))
)
mcols(cis.eqtl.ranges) <- cis.eqtl[,c("gene", "TSS", "beta", "pvalue")]

In [27]:
cis.eqtl.ranges

GRanges object with 1560934 ranges and 4 metadata columns:
            seqnames    ranges strand |            gene       TSS       beta
               <Rle> <IRanges>  <Rle> |     <character> <integer>  <numeric>
        [1]        1 169699715      * | ENSG00000000457 169894267 -0.0495816
        [2]        1 169700002      * | ENSG00000000457 169894267 -0.0506340
        [3]        1 169700591      * | ENSG00000000457 169894267  0.0550592
        [4]        1 169701315      * | ENSG00000000457 169894267  0.0559752
        [5]        1 169701799      * | ENSG00000000457 169894267 -0.0502370
        ...      ...       ...    ... .             ...       ...        ...
  [1560930]        9 125750069      * | ENSG00000287782 125426169  0.0281308
  [1560931]        9 125760087      * | ENSG00000287782 125426169  0.0280961
  [1560932]        9 125760246      * | ENSG00000287782 125426169  0.0280961
  [1560933]        9 125824786      * | ENSG00000287782 125426169  0.0277303
  [1560934]      

In [28]:
overlaps <- findOverlaps(cis.eqtl.ranges, da.peak.ranges)

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 23
  - in 'y': X, KI270726.1, KI270742.1, KI270717.1, GL000008.2, KI270734.1, KI270706.1, GL000221.1, KI270731.1, KI270749.1, KI270733.1, GL000220.1, GL000009.2, GL000194.1, KI270709.1, KI270750.1, GL000195.1, KI270727.1, KI270748.1, Y, GL000224.1, KI270744.1, GL000216.2, GL000219.1, KI270721.1, KI270719.1, KI270712.1, KI270725.1, KI270722.1, KI270707.1, KI270538.1, KI270442.1, KI270708.1, KI270732.1, GL000205.2, KI270737.1, KI270735.1, GL000214.1, KI270438.1, GL000218.1, KI270746.1, KI270736.1
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”


In [29]:
overlap.set <- dplyr::bind_cols(
    as.data.frame(cis.eqtl.ranges[overlaps@from, ]) %>%
        dplyr::select(SNP.Chr=seqnames, SNP.Pos=start, eQTL.Gene=gene, eQTL.TSS=TSS, eQTL.Beta=beta, eQTL.P=pvalue),
    as.data.frame(da.peak.ranges[overlaps@to, ]) %>%
        dplyr::select(Peak.ID=Peak.ID, Peak.Chr=seqnames, Peak.Start=start, Peak.End=end, Peak.Width=width, Peak.Cell.Type=Cell.Type, Peak.Log.FC=logFC, Peak.P=adj.P.Val)
)

There are 3124 unique peaks that have a significant eSNP.

In [30]:
length(unique(overlap.set$Peak.ID))

[1] 11596

There are 9665 eSNPs that overlap with at least one DA peak.

In [31]:
length(unique(paste0(overlap.set$SNP.Chr, ":", overlap.set$SNP.Pos)))

[1] 26170

There are 3958 eGenes that have at least one eSNP present in at least one DA peak.

In [32]:
length(unique(overlap.set$eQTL.Gene))

[1] 6438

In [33]:
da.peak.freqs <- as.data.frame(table(da.peaks$Cell.Type)) %>%
    dplyr::select(Cell.Type=1, DA.Peak.Count=2)

overlap.set.peaks <- overlap.set %>%
    dplyr::select(Peak.ID, Peak.Cell.Type) %>%
    unique()
overlap.freqs <- as.data.frame(table(overlap.set.peaks$Peak.Cell.Type)) %>%
    dplyr::select(Cell.Type=1, Overlap.Count=2) 

merge(da.peak.freqs, overlap.freqs, by="Cell.Type") %>%
    dplyr::mutate(Proportion=Overlap.Count / DA.Peak.Count) %>%
    dplyr::arrange(desc(Proportion))

Cell.Type,DA.Peak.Count,Overlap.Count,Proportion
<fct>,<int>,<int>,<dbl>
Gamma_delta_T,1519,382,0.2514812
CD8pos_T,4923,1183,0.2403006
Effector_CD4pos_T,12428,2982,0.2399421
Naive_CD8_T,1369,326,0.2381300
Effector_memory_CD8pos_T,6916,1613,0.2332273
Memory_Tregs,3253,755,0.2320935
Bulk_B,651,151,0.2319508
Follicular_T_Helper,14246,3207,0.2251158
Regulatory_T,6506,1447,0.2224101


## Overlap of Sentinel cis-eQTL (Conditional Analysis)

I can combine the sentinel SNP at each locus with SNPs identified after conditional analysis (conditioning on the sentinel SNP to identify multiple signals).

In [34]:
da.peak.ranges = GRanges(
    seqnames=da.peaks$Peak.Chr,
    ranges=IRanges(
        start=da.peaks$Peak.Start,
        end=da.peaks$Peak.End
    ),
    strand=rep("*", nrow(da.peaks))
)
mcols(da.peak.ranges) <- da.peaks[,c("Peak.ID", "Cell.Type", "logFC", "adj.P.Val")]

In [35]:
da.peak.ranges

GRanges object with 171830 ranges and 4 metadata columns:
           seqnames              ranges strand |                Peak.ID
              <Rle>           <IRanges>  <Rle> |            <character>
       [1]       13   50244295-50246227      * |   13:50244295-50246227
       [2]       13   79450967-79452794      * |   13:79450967-79452794
       [3]        9   99470831-99473127      * |    9:99470831-99473127
       [4]       16   19293486-19294933      * |   16:19293486-19294933
       [5]       10   90977043-90978869      * |   10:90977043-90978869
       ...      ...                 ...    ... .                    ...
  [171826]       13 112116343-112116882      * | 13:112116343-112116882
  [171827]       16   67758899-67760054      * |   16:67758899-67760054
  [171828]       16   85514268-85516322      * |   16:85514268-85516322
  [171829]        5   94983466-94984498      * |    5:94983466-94984498
  [171830]       22   26614942-26615897      * |   22:26614942-26615897
      

In [36]:
cis.eqtl.sentinel <- dplyr::bind_rows(
    cis.eqtl %>%
        dplyr::group_by(gene) %>%
        dplyr::slice_min(order_by=pvalue, n=1),
    cis.eqtl.conditional
)

cis.eqtl.sentinel.ranges = GRanges(
    seqnames=cis.eqtl.sentinel$chr,
    ranges=IRanges(
        start=cis.eqtl.sentinel$SNPpos,
        end=cis.eqtl.sentinel$SNPpos
    ),
    strand=rep("*", nrow(cis.eqtl.sentinel))
)
mcols(cis.eqtl.sentinel.ranges) <- cis.eqtl.sentinel[,c("gene", "beta", "pvalue")]

In [37]:
cis.eqtl.sentinel.ranges

GRanges object with 33753 ranges and 3 metadata columns:
          seqnames    ranges strand |            gene       beta      pvalue
             <Rle> <IRanges>  <Rle> |     <character>  <numeric>   <numeric>
      [1]        1 169776143      * | ENSG00000000457  0.0498587 5.19259e-12
      [2]        1 169776312      * | ENSG00000000457  0.0498587 5.19259e-12
      [3]        1 169749095      * | ENSG00000000460 -0.0721270 5.42668e-09
      [4]        1 196708768      * | ENSG00000000971  0.1713537 7.51523e-06
      [5]        1 196726612      * | ENSG00000000971  0.1713537 7.51523e-06
      ...      ...       ...    ... .             ...        ...         ...
  [33749]        4  81171813      * | ENSG00000138669 -0.0762586 1.35966e-10
  [33750]        4  76196421      * | ENSG00000138760  0.0766065 4.62452e-08
  [33751]        4 163514546      * | ENSG00000273449  0.0846128 2.45511e-23
  [33752]        4    121772      * | ENSG00000275426 -0.0904912 8.08253e-18
  [33753]        1 

In [38]:
overlaps <- findOverlaps(cis.eqtl.sentinel.ranges, da.peak.ranges)

Warning message in .Seqinfo.mergexy(x, y):
“Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': 23
  - in 'y': X, KI270726.1, KI270742.1, KI270717.1, GL000008.2, KI270734.1, KI270706.1, GL000221.1, KI270731.1, KI270749.1, KI270733.1, GL000220.1, GL000009.2, GL000194.1, KI270709.1, KI270750.1, GL000195.1, KI270727.1, KI270748.1, Y, GL000224.1, KI270744.1, GL000216.2, GL000219.1, KI270721.1, KI270719.1, KI270712.1, KI270725.1, KI270722.1, KI270707.1, KI270538.1, KI270442.1, KI270708.1, KI270732.1, GL000205.2, KI270737.1, KI270735.1, GL000214.1, KI270438.1, GL000218.1, KI270746.1, KI270736.1
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning).”


In [39]:
overlap.set <- dplyr::bind_cols(
    as.data.frame(cis.eqtl.sentinel.ranges[overlaps@from, ]) %>%
        dplyr::select(SNP.Chr=seqnames, SNP.Pos=start, eQTL.Gene=gene, eQTL.Beta=beta, eQTL.P=pvalue),
    as.data.frame(da.peak.ranges[overlaps@to, ]) %>%
        dplyr::select(Peak.ID=Peak.ID, Peak.Chr=seqnames, Peak.Start=start, Peak.End=end, Peak.Width=width, Peak.Cell.Type=Cell.Type, Peak.Log.FC=logFC, Peak.P=adj.P.Val)
)

There are 309 unique peaks that have a sentinel eSNP.

In [40]:
length(unique(overlap.set$Peak.ID))

[1] 674

There are 375 sentinel eSNPs that overlap with at least one DA peak.

In [41]:
length(unique(paste0(overlap.set$SNP.Chr, ":", overlap.set$SNP.Pos)))

[1] 787

There are 348 eGenes that have at least one eSNP present in at least one DA peak.

In [42]:
length(unique(overlap.set$eQTL.Gene))

[1] 693